# 📊 Análisis exploratorio: 🏦 Bank Marketing Dataset (UCI)

## 1️⃣ **Introducción y contexto del proyecto**

El análisis exploratorio que se va a realizar está basado en el dataset **Bank Marketing**, publicado por el Banco de Portugal y disponible en el repositorio UCI Machine Learning:  
🔗 https://archive.ics.uci.edu/dataset/222/bank+marketing  

Este conjunto de datos recoge información de campañas de marketing telefónico realizadas por una entidad bancaria portuguesa. El banco contactaba a clientes potenciales con el fin de ofrecerles un depósito a plazo fijo. Cada fila representa a un cliente contactado y las variables recogen aspectos demográficos, financieros y operativos del proceso de captación. Los datos recogidos en este dataset están ordenadores por fecha, desde Mayo del 2008 a Noviembre del 2010.

El fichero que analizaremos, **bank-full.csv**, contiene **45.211 registros** y **17 variables** relacionadas con datos demográficos, laborales, financieros y características de la interacción comercial.

Aunque el objetivo original del dataset era predecir si un cliente contrataría un depósito a plazo fijo, en este proyecto vamos a reutilizar estas variables para analizar y modelar dos grandes escenarios reales dentro del sector financiero: **gestión de patrimonio** y **gestión del riesgo**.

---

## 🎯 **Objetivo del análisis**

El propósito de este análisis es **comprender el perfil financiero y sociodemográfico de los clientes del banco**, analizando sus patrones económicos y su posible estabilidad o capacidad de ahorro.

Para ello, exploraremos en profundidad la columna `balance`, que refleja el **saldo anual promedio de la cuenta bancaria del cliente**. A partir de esta métrica construiremos **dos posibles variables objetivo (targets)**, inspiradas en escenarios reales del sector bancario:

---


## 🅰️ **Escenario A: El Umbral del 90% — Gestión de Patrimonio**

En banca privada existe un fenómeno conocido: **el 10% de los clientes más ricos generan alrededor del 80% de los ingresos del banco** (Principio de Pareto). Identificar este grupo es clave para priorizar servicios personalizados y productos de alto valor.

**Acción realista:**  
Calcular el **percentil 90 (P90)** del `balance`. Ese valor representa el **mínimo saldo que posee el 10% de clientes más adinerados**.

A partir de ese umbral crearemos la variable binaria `Patrimonio_Alto`:

- `Patrimonio_Alto = 1` si `balance > P90`  
- `Patrimonio_Alto = 0` si `balance ≤ P90`

**Interpretación práctica:**  
El modelo resultante permitirá estimar si un cliente **pertenece al segmento de alto patrimonio**, a partir de su edad, trabajo, educación, historial crediticio y comportamiento financiero.  
Esto imita cómo las divisiones de **Wealth Management** detectan clientes con potencial para productos premium.

---

## 🅱️ **Escenario B: El Umbral Cero — Riesgo y Sobregiro**

En gestión de riesgos bancarios, uno de los indicadores más críticos es detectar **clientes con saldo negativo** o riesgo de entrar en sobregiro. Un balance por debajo de cero señala inestabilidad financiera o dependencia del crédito.

**Acción realista:**  
Utilizar el valor **0 como umbral**.

Crearemos la variable binaria `Riesgo_Sobregiro`:

- `Riesgo_Sobregiro = 1` si `balance < 0`  
- `Riesgo_Sobregiro = 0` si `balance ≥ 0`

**Interpretación práctica:**  
El modelo predecirá qué clientes tienen una **alta probabilidad de estar en números rojos**, lo que resulta esencial para los departamentos de **Risk Management**.  
Esto ayuda a anticipar comportamientos financieros que puedan derivar en impagos o necesidad de intervención.

---

## 🧩 **Significado de las variables del dataset**

| **Columna** | **Descripción** |
|-------------|-----------------|
| `age` | Edad del cliente. |
| `job` | Tipo de trabajo u ocupación. |
| `marital` | Estado civil. |
| `education` | Nivel de educación alcanzado. |
| `default` | ¿Tiene impagos en créditos anteriores? |
| `balance` | Saldo medio anual de la cuenta bancaria. |
| `housing` | ¿Tiene hipoteca en curso? (yes/no) |
| `loan` | ¿Tiene algún préstamo personal? |
| `contact` | Tipo de contacto empleado (teléfono móvil o fijo). |
| `day` | Día del mes en que se realizó el último contacto. |
| `month` | Mes del último contacto. |
| `duration` | Duración (en segundos) de la última llamada. |
| `campaign` | Número de contactos realizados durante la campaña. |
| `pdays` | Días desde el último contacto previo a esta campaña. |
| `previous` | Número de contactos en campañas anteriores. |
| `poutcome` | Resultado de campañas anteriores. |
| `y` | Variable original del banco: ¿suscribió el depósito a plazo? (yes/no). |

---

📌 *Fuente de los datos: UCI Machine Learning Repository, Bank Marketing Dataset (2012).*  

Este análisis nos permitirá comprender mejor los perfiles financieros de los clientes y construir modelos predictivos aplicables a contextos reales de banca, tanto en **gestión de patrimonio** como en **evaluación de riesgo**.

---

## 2️⃣ **Importación de librerías y carga del dataset**

Incluye todas las librerías necesarias para manipular datos, visualizar resultados y entrenar modelos.  
Se carga también el fichero `bank-full.csv` y se muestran sus primeras filas para verificar que se ha importado correctamente.

In [ ]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
import io # Agregamos io para el manejo de strings como archivos

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y modelos (scikit-learn)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos que usaremos más adelante
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# --- Modelos de Regresión Lineal ---
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# --- Modelos Basados en Árboles ---
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# --- Métricas de Evaluación ---
from sklearn.metrics import r2_score, mean_squared_error

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

import time
import numpy as np
from sklearn.metrics import f1_score

Cargamos el dataset y visualizamos las primeras filas

In [ ]:
dataset = './data/bank-full.csv'

# 1. Leer el archivo completo como texto
try:
    with open(dataset, 'r', encoding='utf-8') as f:
        content = f.read()
except FileNotFoundError:
    # Manejo de error si el archivo no está en la ubicación esperada
    print(f"Error: No se encontró el archivo '{dataset}'. Asegúrate de que está cargado correctamente.")
    exit()

# 2. Modificación solicitada: Reemplazar todas las comillas dobles (") con una cadena vacía
# Esto limpia tanto el encabezado como el resto de los datos.
cleaned_content = content.replace('"', '')

# 3. Leer el DataFrame usando io.StringIO
# io.StringIO permite a Pandas leer la cadena de texto como si fuera un archivo.
df = pd.read_csv(io.StringIO(cleaned_content), sep=';')

df.head()

---

## 3️⃣ **Exploración inicial del dataset (EDA básico)**

Este apartado da las primeras pistas sobre posibles problemas o particularidades del dataset.

- .info(), .describe(), .nunique()
- Dimensiones, tipos de datos, distribución inicial.

In [ ]:
# Dimensiones del dataset
df.shape

In [ ]:
# Información general del dataset
df.info()

In [ ]:
# Estadísticas generales
df.describe()

In [ ]:
# Número de valores únicos por columna
df.nunique()

In [ ]:
# Revisamos la distribución inicial de la variable objetivo para saber si el dataset está desbalanceado
df['balance'].value_counts()

---

## 4️⃣ **Limpieza de datos**
En esta parte se revisa y corrige la calidad de los datos:
- Detección de valores nulos  
- Comprobación de categorías inconsistentes  
- Revisión de posibles valores atípicos extremos que distorsionen el análisis  
El propósito es garantizar que el dataset sea adecuado para construir modelos fiables.

In [ ]:
# Revisamos los valores nulos que pueda tener el dataset
print("\n--- VALORES NULOS ---")
df.isnull().sum()

In [ ]:
print("\n--- VALORES DUPLICADOS ---")
print(df.duplicated().sum())

In [ ]:
# CAMBIAMOS EL NOMBRE A LA VARIABLE 'y' QUE ERA LA VARIABLE OBJETIVO ORIGINAL DEL DATASET
# Definimos el cambio de nombre
rename_map = {
    'y': 'subscribed_term_deposit'
}

# Aplicamos el cambio de nombre
df = df.rename(columns=rename_map)

# Mostramos las primeras filas y los nombres de las columnas para verificar
print("Nombres de columnas después de renombrar:")
print(df.columns.tolist())

# BORRAR COLUMNAS QUE APORTEN RUIDO A LOS MODELOS, EN LA EXPLICACIÓN PONER QUE SE HAN HECHO PRUEBAS CON ESAS COLUMNAS Y SIN ELLAS PARA DEMOSTRAR QUE NO APORTAN NADA

# Dataframe con columnas borradas

In [ ]:
df2 = df.copy()

df2.drop(columns=['duration', 'day', 'pdays', 'previous', 'poutcome', 'contact'], inplace=True)

---

## 5️⃣ **Análisis profundo del variable `balance` (variable clave del proyecto)**
La columna `balance` es el pilar de las dos variables objetivo.  
En este apartado se debe:
- Analizar la distribución del balance  
- Identificar valores negativos, picos extraños o colas largas  
- Calcular rangos, percentiles clave (P50, P75, P90, P95)  
- Visualizarlo con histogramas, boxplots o KDE  
Este análisis justifica los umbrales propuestos para los dos escenarios financieros.

### 5.1. Revisión inicial y distribución general

In [ ]:
# Primero verificamos el tipo de la columna 'balance' y visualizamos los primeros valores
print("---------- TIPO DE LA COLUMNA BALANCE ----------")
print(df['balance'].dtype)

In [ ]:
print("---------- MOSTRAMOS LAS 5 PRIMERAS FILAS DE LA COLUMNA BALANCE ----------")
df['balance'].head()

In [ ]:
print("---------- MOSTRAMOS DATOS DESCRIPTIVOS DE LA COLUMNA BALANCE")
df['balance'].describe()

#### ¿Qué conclusiones podemos sacar con estos valores?

Al analizar la columna balance, vemos una realidad financiera bastante desigual entre los clientes del banco. Para empezar, encontramos balances negativos que llegan hasta –8.019 €, lo que nos confirma que hay personas que están en descubierto o arrastran deudas importantes. Esto ya nos da una pista clara: hay un grupo de clientes que podría considerarse en una situación de riesgo, y merece la pena calcular qué parte del dataset se encuentra en números rojos.

Si miramos los percentiles, también queda bastante claro que la mayoría de los clientes no tiene grandes cantidades de dinero en la cuenta. El 25% no llega ni a 72 €, la mitad se queda alrededor de los 448 €, y el 75% está por debajo de 1.428 €. Dicho de forma sencilla: la gran mayoría de los clientes tiene menos de 1.500 € en balance, lo que apunta a perfiles con poco margen financiero y un nivel de ahorro limitado. Además, como la mediana (448 €) es muy inferior a la media (1.362 €), podemos intuir que hay unos pocos clientes con mucho dinero que están tirando del promedio hacia arriba.

Y esto se confirma cuando miramos el valor máximo: 102.127 €, que es una cifra enorme comparada con lo que tiene la mayoría. Esto nos muestra una distribución con una cola derecha muy larga, típica cuando unos pocos clientes acumulan mucho capital. Este pequeño grupo de “grandes balances” hace que la desviación estándar sea muy alta (3.044 €), reflejando que las diferencias entre clientes son bastante grandes.

En resumen, el comportamiento del balance no es uniforme ni equilibrado: hay muchas personas con poco dinero o incluso en negativo, y unas pocas con cantidades muy altas. Por eso, para definir umbrales que clasifiquen a los clientes (por ejemplo: “riesgo”, “estable”, “alto balance”), tiene más sentido apoyarse en percentiles que en la media. La mediana y los percentiles superiores (como P90 o P95) cuentan una historia mucho más fiel de lo que realmente ocurre en la base de clientes.

### **5.2. Analizar específicamente los valores negativos**

En este apartado lo que queremos es:

- Saber cuántos clientes están en negativo
- Calcular qué porcentaje representan
- Ver hasta qué punto pueden ser negativos
- Entender si hay valores extremos (outliers negativos)
- Detectar si están agrupados o dispersos

In [ ]:
# Primero filtramos los valores negativos
negativos = df[df['balance'] < 0]

# Calculamos el total de valores negativos
cantidad_negativos = len(negativos)
print(f"Cantidad total de filas con un 'balance' negativo: {cantidad_negativos}")

# Calculamos el porcentaje que representa sobre el total
porcentaje_negativos = (cantidad_negativos / len(df)) * 100
print(f"Porcentaje de negativos en el dataset: {round(porcentaje_negativos, 4)}%")

# Miramos también cuál es el valor mínimo y el máximo dentro del grupo de valores negativos
min_negativo = negativos['balance'].min()
max_negativo = negativos['balance'].max()

print(f"Valor mínimo dentro del grupo de negativos: {min_negativo}")
print(f"Valor máximo dentro del grupo de negativos: {max_negativo}")

In [ ]:
# Visualizamos con un gráfico la distribución de valores negativos para visualizar si los valores negativos se concentran cerca de cero o si hay muchos casos extremos muy negativos
plt.figure(figsize=(20, 4))

sns.violinplot(
    x=negativos['balance'],
    orient='h',
    inner='quartile',
    color='#6EC6B0',      # color más profesional (teal suave)
    linewidth=1.2,        # borde más definido
    cut=0
)

plt.title("Distribución de balances negativos")
plt.xlabel("Balance (€)")

# Crear ticks de 0 a -8000 cada 500
plt.xticks(
    ticks=range(0, -8500, -500), 
    labels=[str(t) for t in range(0, -8500, -500)]
)

plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))

sns.violinplot(
    x=negativos['balance'],
    orient='h',
    inner='quartile',
    color='#6EC6B0',      # color más profesional (teal suave)
    linewidth=1.2,        # borde más definido
    cut=0
)

plt.title("Distribución de balances negativos (Zoom 0 a –1500)", fontsize=14)
plt.xlabel("Balance (€)", fontsize=12)

# Crear ticks de 0 a -1500 cada 500 (zona donde está el 90% de los valores negativos)
ticks = range(0, -1600, -100)
labels = [f"{t} €" if t == 0 else f"–{abs(t)} €" for t in ticks]

plt.xticks(
    ticks=ticks,
    labels=labels,
    fontsize=11
)

# Activamos grid principal
plt.grid(axis='x', linestyle='--', alpha=0.6, linewidth=0.7)

# Activamos grid menor para más guía visual
plt.minorticks_on()
plt.grid(axis='x', which='minor', linestyle=':', alpha=0.3)

# Zoom en la parte donde se concentra el 90% de los valores negativos
plt.xlim(-1500, 0)

plt.tight_layout()
plt.show()


### **¿Conclusiones?**

Los dos gráficos muestran que la mayoría de los clientes que están en negativo lo están por poco. La mayor concentración aparece cerca de 0 €, especialmente entre 0 y –300 €, lo que indica que la mayoría de los descubiertos son pequeños y probablemente temporales.

A medida que el balance se vuelve más negativo, la distribución se va estrechando, lo que refleja que cada vez hay menos clientes con deudas medias (entre –500 € y –1500 €). Son casos menos frecuentes y bastante dispersos.

En el gráfico completo se aprecia claramente que los valores extremadamente negativos existen pero son muy poco comunes. La enorme diferencia entre la concentración cercana a cero y la cola larga hacia –8000 € confirma que los descubiertos profundos son excepcionales y que el grupo principal de clientes en negativo tiene un perfil de riesgo mucho más moderado.

### 5.3. Calcular percentiles clave (P50, P75, P90 y P95)

In [ ]:
# ==== Cálculo de percentiles clave ====
p50 = np.percentile(df['balance'], 50)
p75 = np.percentile(df['balance'], 75)
p90 = np.percentile(df['balance'], 90)
p95 = np.percentile(df['balance'], 95)

print("Percentil 50 (mediana):", p50)
print("Percentil 75:", p75)
print("Percentil 90:", p90)
print("Percentil 95:", p95)

Tras calcular los percentiles clave de la columna balance, observamos que el 50% de los clientes tiene 448 € o menos, lo que describe claramente al cliente promedio del banco. El percentil 75 se sitúa en torno a 1428 €, indicando que solo una cuarta parte del conjunto supera este nivel.

Los percentiles altos (P90 ≈ 3574 € y P95 ≈ 5768 €) confirman que los clientes con balances altos representan un porcentaje muy reducido.

En la banca real, no todos los clientes son iguales. El 10% de los clientes más ricos a menudo generan el 80% de los ingresos de un banco ("Principio de Pareto"), por lo que elegiremos utilizar el valor del percentil 90.

### **5.4. Representación gráfica mediante Histograma, KDE y Boxplot**

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(df['balance'], bins=60, kde=True, color='#4A90E2')
plt.title("Distribución completa de la variable balance")
plt.xlabel("Balance (€)")
plt.ylabel("Frecuencia")
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))

sns.histplot(
    df[(df['balance'] >= -3000) & (df['balance'] <= 3000)]['balance'],
    bins=60,
    kde=True,
    color='#7E57C2'
)

plt.title("Distribución del balance (Zoom –3000€ a 3000€)")
plt.xlabel("Balance (€)")
plt.ylabel("Frecuencia")
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.show()


# **ESTA PARTE HAY QUE REVISAR LOS GRÁFICOS QUE NO ME CONVENCEN MUCHO**

### ¿Conclusiones?

# FASFSASGFASGAG

---

## 6️⃣ **Análisis exploratorio de variables financieras y de riesgo**

- Distribución de edades
- Clientes con default (impagos)
- Préstamos personales (loan)
- Personas con hipoteca (housing)

### **6.1. Distribución de edad de la cartera de clientes**

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df["age"], kde=True, bins=30)
plt.title("Distribución de la edad de los clientes")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.show()

# Boxplot para ver outliers
plt.figure(figsize=(6,4))
sns.boxplot(x=df["age"])
plt.title("Boxplot de la edad")
plt.show()

### **6.2. Distribución de clientes con impagos anteriores**

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="default")
plt.title("Clientes con impagos previos (default)")
plt.xlabel("default")
plt.ylabel("Cantidad")
plt.show()

### **6.3. Distribución de clientes con préstamos personales**

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="loan", palette="viridis")
plt.title("¿El cliente tiene préstamo personal?")
# plt.xlabel("¿Tienes un préstamo")
plt.ylabel("Cantidad de clientes")
plt.show()

### **6.4. Distribución de clientes que tienen una hipoteca en curso**

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="housing")
plt.title("Clientes con préstamo de vivienda")
# plt.xlabel("housing")
plt.ylabel("Cantidad")
plt.show()

### **6.5. Matriz de correlación de variables numéricas**

In [ ]:
df_encoded = df.copy()

df_encoded = df_encoded.drop(columns=['pdays', 'contact', 'month', 'day', 'poutcome', 'job', 'duration', 'campaign', 'previous'])

df_encoded = pd.get_dummies(df_encoded, columns=['marital', 'education', 'default', 'housing', 'loan', 'subscribed_term_deposit'], drop_first=True)

df_encoded.columns.tolist()

In [ ]:
# Calculamos la matriz de correlación
corr_matrix = df_encoded.corr()

# Graficamos el heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(
    corr_matrix, 
    annot=True,      # Muestra los números (coeficientes) en cada celda
    fmt='.2f',       # Formatea los números a 2 decimales
    cmap='coolwarm',
    linewidths=0.5
)
plt.title('Matriz de Correlación de Variables Relevantes')
plt.show()

## **CONCLUSIONES**



---

## 7️⃣ **Creación de los dos escenarios (targets)**

En ambos casos se debe analizar qué proporción de clientes pertenece a cada grupo y si es necesario aplicar técnicas para tratar desbalances.

### 🟩 Escenario A: `Patrimonio_Alto`
##### 1. Crear la variable binaria según el umbral

In [ ]:
# Crear la variable binaria 'Patrimonio_Alto'

# La variable podrá tener el valor 1 si el balance es mayor que el umbral, 0 si no lo es.
df["Patrimonio_Alto"] = (df["balance"] > p90).astype(int)

# Comprobar la creación de la nueva columna
print("Primeras filas con la nueva variable 'Patrimonio_Alto':")
print(df[['balance', 'Patrimonio_Alto']].head(10))

##### 2. Comprobar cuántos registros quedan en cada clase (balanceadas/desbalanceadas)

In [ ]:
# Contar el número de registros en cada clase
conteo_clases = df['Patrimonio_Alto'].value_counts()
proporcion_clases = df['Patrimonio_Alto'].value_counts(normalize=True) * 100

print("\nResultados del Análisis de Desbalance de 'Patrimonio_Alto':")
print("---------------------------------------------------------")
print(f"Conteo de Clases:\n{conteo_clases}")
print("\nProporción de Clases:")
print(proporcion_clases.map('{:.2f}%'.format))

Gracias a esto observamos un dataset claramente desbalanceado, como era de esperar porque P90 define al 10% superior

### 🟥 Escenario B: `Riesgo_Sobregiro`
- Aplicar el umbral cero  
- Crear la variable binaria correspondiente  
- Revisar la distribución resultante 

##### 1. Aplicar el umbral cero y crear la variable binaria según el umbral

In [ ]:
# Definimos el umbral 0
umbral_cero = 0

# Creamos la variable binaria 'Riesgo_Sobregiro'
# 1 si el balance es menor que cero (riesgo/sobregiro), 0 si es cero o positivo.
df['Riesgo_Sobregiro'] = np.where(df['balance'] < umbral_cero, 1, 0)

# Comprobamos la creación de la nueva columna
print("Primeras filas con la nueva variable 'Riesgo_Sobregiro':")
print(df[['balance', 'Riesgo_Sobregiro']].head(10))

##### 2. Comprobar cuántos registros quedan en cada clase (balanceadas/desbalanceadas)

In [ ]:
# Contar el número de registros en cada clase
conteo_clases_riesgo = df['Riesgo_Sobregiro'].value_counts()
proporcion_clases_riesgo = df['Riesgo_Sobregiro'].value_counts(normalize=True) * 100

print("\nResultados del Análisis de Desbalance de 'Riesgo_Sobregiro':")
print("----------------------------------------------------------")
print(f"Conteo de Clases:\n{conteo_clases_riesgo}")
print("\nProporción de Clases:")
print(proporcion_clases_riesgo.map('{:.2f}%'.format))

### ¿Conclusiones?

Al crear el escenario **Patrimonio_Alto**, observamos que únicamente el 10% de los clientes supera el umbral del percentil 90 del balance. Esto confirma que el grupo de clientes con un nivel económico claramente elevado es muy reducido en comparación con el resto de la base. La distribución está fuertemente desbalanceada, lo que significa que cualquier modelo que utilice esta variable necesitará tener en cuenta este desequilibrio para no favorecer en exceso a la clase mayoritaria.

En el escenario **Riesgo_Sobregiro**, los resultados muestran que aproximadamente el 8.33% de los clientes tiene saldo negativo, mientras que el 91.67% mantiene un balance igual o superior a cero. Aunque este escenario también presenta desbalance, la diferencia entre clases es ligeramente menor que en el caso del patrimonio alto. Aun así, sigue siendo importante considerar técnicas de balanceo si se desea entrenar modelos de clasificación consistentes.

En conjunto, ambos escenarios revelan una estructura muy desigual en los extremos financieros de la cartera: solo una minoría acumula altos niveles de patrimonio y una minoría diferente incurre en sobregiro. Esto ofrece oportunidades interesantes para el modelado, pero también exige atención especial a la desproporción entre clases que podría afectar a futuros algoritmos de machine learning.

---

## 8️⃣ **Preprocesamiento para Machine Learning**
Aquí se preparan los datos antes de entrenar modelos:
- Identificar variables categóricas y numéricas  
- Crear transformadores:  
  - `OneHotEncoder` para categorías  
  - `StandardScaler` para variables numéricas  
- Separar en train/test  

#### 1. Separar datos en X e Y

In [ ]:
X = df.drop(columns=['Patrimonio_Alto', 'Riesgo_Sobregiro', 'balance'])

# Variable objetivo para el escenario A
y_escenario_A = df['Patrimonio_Alto']

# Variable objetivo para el escenario B
y_escenario_B = df['Riesgo_Sobregiro']

X e Y dataframe con columnas eliminadas:

In [ ]:
X2 = df2.drop(columns=['Patrimonio_Alto', 'Riesgo_Sobregiro', 'balance'])

# Variable objetivo para el escenario A
y_escenario_A2 = df2['Patrimonio_Alto']

# Variable objetivo para el escenario B
y_escenario_B = df2['Riesgo_Sobregiro']

#### 2. Identificar variables categóricas y numéricas

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns

print("Numéricas:", list(num_features))
print("Categóricas:", list(cat_features))

Numericas y categoricas para dataframe con columnas eliminadas:

In [ ]:
num_features = X2.select_dtypes(include=['int64', 'float64']).columns
cat_features = X2.select_dtypes(include=['object']).columns

print("Numéricas:", list(num_features))
print("Categóricas:", list(cat_features))

#### 3. Crear transformadores

Para las categóricas usaremos OneHotEncoder y para las numéricas StandardScaler

In [ ]:
# Categóricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Numéricas
numeric_transformer = StandardScaler()

#### 4. Crear el ColumnTransformer

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)

## 🟩 Escenario A — `Patrimonio_Alto`

#### 5. Dividir en train/test

In [ ]:
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(
    X,
    y_escenario_A,
    test_size=0.2,
    random_state=42,
    stratify=y_escenario_A
)

#### 6. Aplicar el preprocesamiento

In [ ]:
from imblearn.over_sampling import SMOTE

# Ajustar el preprocesador solo con los datos de entrenamiento
X_train_A_preprocessed = preprocessor.fit_transform(X_train_A)

# Transformar los datos de test con el mismo ajuste
X_test_A_preprocessed = preprocessor.transform(X_test_A)

REVISAR SE UTILIZA PARA EQUILIBRAR:

In [ ]:
# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_A_preprocessed, y_train_A)

SMOTE

Solo debes aplicarlo al conjunto de entrenamiento, nunca al test.

Funciona solo con variables numéricas → si tienes variables categóricas, deben estar ya codificadas (OneHotEncoder, LabelEncoder, etc.).

## 🟥 Escenario B: `Riesgo_Sobregiro`

#### 5. Dividir en train/test

In [ ]:
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(
    X,
    y_escenario_B,
    test_size=0.2,
    random_state=42,
    stratify=y_escenario_B
)

#### 6. Aplicar el preprocesamiento

In [ ]:
# Ajustar el preprocesador solo con los datos de entrenamiento
X_train_B_preprocessed = preprocessor.fit_transform(X_train_B)

# Transformar los datos de test con el mismo ajuste
X_test_B_preprocessed = preprocessor.transform(X_test_B)

---

## 9️⃣ **Entrenamiento de modelos de clasificación**
Se entrenan modelos como:
- Regresión Logística  
- Árbol de Decisión  
- Random Forest  
- KNN  
- Naive Bayes  
- SVM  
- MLP (Red Neuronal sencilla)

El enfoque es comparativo: ver cuál funciona mejor según las métricas.

## 🟩 Escenario A — `Patrimonio_Alto`

### Regresión logística

In [ ]:
rl_a = LogisticRegression(max_iter=1500)

inicio = time.time()
rl_a.fit(X_train_A_preprocessed, y_train_res)
fin = time.time()

# Realiza predicciones
y_pred_lr = rl_a.predict(X_test_A_preprocessed)

tiempo_rl_a = fin - inicio
print("Tiempo Logistic Regression (Escenario A):")
print(f"{tiempo_rl_a:.2f}")

print(classification_report(y_test_A, y_pred_lr))

# Calcula la matriz de confusión
cm_lr = confusion_matrix(y_test_A, y_pred_lr)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_lr, display_labels=['Clase media/baja (0)', 'Clase alta (1)'])
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:

rl_a = LogisticRegression(max_iter=1500)

inicio = time.time()
rl_a.fit(X_train_res, y_train_res)
fin = time.time()

import numpy as np
from sklearn.metrics import f1_score

# CALCULAR EL UMBRAL ÓPTIMO
# Encontrar el umbral que maximiza el F1 de la clase 1.
y_proba = rl_a.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("Mejor umbral:", best_threshold)
print("Mejor F1:", best_f1)


# Realiza predicciones
y_pred_lr = (y_proba > best_threshold).astype(int)

tiempo_rl_a = fin - inicio
print("Tiempo Logistic Regression (Escenario A):")
print(f"{tiempo_rl_a:.2f}")

print(classification_report(y_test_A, y_pred_lr))

# Calcula la matriz de confusión
cm_lr = confusion_matrix(y_test_A, y_pred_lr)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_lr, display_labels=['Clase media/baja (0)', 'Clase alta (1)'])
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()

### Árbol de decisión optimizado

In [ ]:
tree_opt_a = DecisionTreeClassifier()

# Definir el diccionario de parámetros a probar
param_grid = {
    'max_depth': [None, 3, 5, 7, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=tree_opt_a,
    param_grid=param_grid,
    cv=5,  # validación cruzada con 5 particiones
    scoring='f1',
    n_jobs=-1  # usa todos los núcleos disponibles para acelerar
)

# Entrenar el GridSearchCV
inicio = time.time()
grid_search.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

print(grid_search.best_params_)

# Obtener el mejor modelo
best_model_dt = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_dt_opt = best_model_dt.predict(X_test_A_preprocessed)

tiempo_dt_a = fin - inicio
print("Tiempo Decision Tree Optimizado (Escenario A):")
print(f"{tiempo_dt_a:.2f}")

print(classification_report(y_test_A, y_pred_dt_opt))

cm_dt_opt = confusion_matrix(y_test_A, y_pred_dt_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_dt_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
tree_opt_a = DecisionTreeClassifier()

# Definir el diccionario de parámetros a probar
param_grid = {
    'max_depth': [None, 3, 5, 7, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=tree_opt_a,
    param_grid=param_grid,
    cv=5,  # validación cruzada con 5 particiones
    scoring='f1',
    n_jobs=-1  # usa todos los núcleos disponibles para acelerar
)

# Entrenar el GridSearchCV
inicio = time.time()
grid_search.fit(X_train_res, y_train_res)
fin = time.time()

print(grid_search.best_params_)

# Obtener el mejor modelo
best_model_dt = grid_search.best_estimator_

y_proba = best_model_dt.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("Mejor umbral:", best_threshold)
print("Mejor F1:", best_f1)

# Predecir en el conjunto de prueba
y_pred_dt_opt = (y_proba > best_threshold).astype(int)

tiempo_dt_a = fin - inicio
print("Tiempo Decision Tree Optimizado (Escenario A):")
print(f"{tiempo_dt_a:.2f}")

print(classification_report(y_test_A, y_pred_dt_opt))

cm_dt_opt = confusion_matrix(y_test_A, y_pred_dt_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_dt_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()

### Random Forest

In [ ]:
random_tree_a = RandomForestClassifier()

inicio = time.time()
random_tree_a.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

y_pred_rf = random_tree_a.predict(X_test_A_preprocessed)

tiempo_rt_a = fin - inicio
print("Tiempo Random Forest (Escenario A):")
print(f"{tiempo_rt_a:.2f}")

print(classification_report(y_test_A, y_pred_rf))

cm_rt = confusion_matrix(y_test_A, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Random Forest")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
random_tree_a = RandomForestClassifier()

inicio = time.time()
random_tree_a.fit(X_train_res, y_train_res)
fin = time.time()

y_proba = random_tree_a.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

y_pred_rf = (y_proba > best_threshold).astype(int)

tiempo_rt_a = fin - inicio
print("Tiempo Random Forest (Escenario A):")
print(f"{tiempo_rt_a:.2f}")

print(classification_report(y_test_A, y_pred_rf))

cm_rt = confusion_matrix(y_test_A, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Random Forest")
plt.show()

### Random Forest Optimizado

In [ ]:
random_tree_opt = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=random_tree_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_rf = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_rf_opt = best_model_rf.predict(X_test_A_preprocessed)

tiempo_rf_opt_a = fin - inicio
print("Tiempo Random Forest Optimizado (Escenario A):")
print(f"{tiempo_rf_opt_a:.2f}")

print(classification_report(y_test_A, y_pred_rf_opt))

cm_rf_opt = confusion_matrix(y_test_A, y_pred_rf_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Random Forest Optimizado")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
random_tree_opt = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=random_tree_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_res, y_train_res)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_rf = grid_search.best_estimator_

y_proba = random_tree_a.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Predecir en el conjunto de prueba
y_pred_rf_opt = (y_proba > best_threshold).astype(int)

tiempo_rf_opt_a = fin - inicio
print("Tiempo Random Forest Optimizado (Escenario A):")
print(f"{tiempo_rf_opt_a:.2f}")

print(classification_report(y_test_A, y_pred_rf_opt))

cm_rf_opt = confusion_matrix(y_test_A, y_pred_rf_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Random Forest Optimizado")
plt.show()

### KNN

In [ ]:
knn = KNeighborsClassifier()

inicio = time.time()
knn.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

y_pred_knn = knn.predict(X_test_A_preprocessed)

tiempo_knn_a = fin - inicio

print("Tiempo KNN (Escenario A):")
print(f"{tiempo_knn_a:.2f}")

print(classification_report(y_test_A, y_pred_knn))

cm_knn = confusion_matrix(y_test_A, y_pred_knn)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - KNN")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
knn = KNeighborsClassifier()

inicio = time.time()
knn.fit(X_train_res, y_train_res)
fin = time.time()

y_proba = knn.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

y_pred_knn = (y_proba > best_threshold).astype(int)

tiempo_knn_a = fin - inicio

print("Tiempo KNN (Escenario A):")
print(f"{tiempo_knn_a:.2f}")

print(classification_report(y_test_A, y_pred_knn))

cm_knn = confusion_matrix(y_test_A, y_pred_knn)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - KNN")
plt.show()

### KNN Optimizado

In [ ]:
knn_opt = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [1, 5, 10, 15, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=knn_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_knn = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_knn_opt = best_model_knn.predict(X_test_A_preprocessed)

tiempo_knn_opt_a = fin - inicio
print("Tiempo KNN Optimizado (Escenario A):")
print(f"{tiempo_knn_opt_a:.2f}")

cm_knn_opt = confusion_matrix(y_test_A, y_pred_knn_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - KNN Optimizado")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
knn_opt = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [1, 5, 10, 15, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=knn_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_res, y_train_res)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_knn = grid_search.best_estimator_

y_proba = best_model_knn.predict_proba(X_test_A_preprocessed)[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Predecir en el conjunto de prueba
y_pred_knn_opt = (y_proba > best_threshold).astype(int)

tiempo_knn_opt_a = fin - inicio
print("Tiempo KNN Optimizado (Escenario A):")
print(f"{tiempo_knn_opt_a:.2f}")

cm_knn_opt = confusion_matrix(y_test_A, y_pred_knn_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn_opt)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - KNN Optimizado")
plt.show()

### Naive Bayes

In [ ]:
nb = GaussianNB()

inicio = time.time()
nb.fit(X_train_A_preprocessed.toarray(), y_train_A)
fin = time.time()

y_pred_nb = nb.predict(X_test_A_preprocessed)

tiempo_nb_a = fin - inicio
print("Tiempo Naive Bayes (Escenario A):")
print(f"{tiempo_nb_a:.2f}")

print(classification_report(y_test_A, y_pred_nb))

cm_nb = confusion_matrix(y_test_A, y_pred_nb)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_nb)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Naive Bayes")
plt.show()

#### Modificado para que este más equilibrado y no salga tan mal por estar tan desbalanceado REVISAR SI SE PUEDE MEJORAR

In [ ]:
nb = GaussianNB()

inicio = time.time()
nb.fit(X_train_res.toarray(), y_train_res)
fin = time.time()

y_proba = nb.predict_proba(X_test_A_preprocessed.toarray())[:, 1]

best_f1 = 0
best_threshold = 0

for t in np.arange(0.01, 1.0, 0.01):
    y_pred_t = (y_proba > t).astype(int)
    f1 = f1_score(y_test_A, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

y_pred_nb = (y_proba > best_threshold).astype(int)

tiempo_nb_a = fin - inicio
print("Tiempo Naive Bayes (Escenario A):")
print(f"{tiempo_nb_a:.2f}")

print(classification_report(y_test_A, y_pred_nb))

cm_nb = confusion_matrix(y_test_A, y_pred_nb)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_nb)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión - Naive Bayes")
plt.show()

### Naive Bayes Optimizado

In [ ]:
nb_opt = GaussianNB()

param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=nb_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_nb = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_nb_opt = best_model_nb.predict(X_test_A_preprocessed)

tiempo_nb_opt_a = fin - inicio

cm_nb_opt = confusion_matrix(y_test_A, y_pred_nb_opt)

### Support Vector Machines

In [ ]:
svc = SVC()

inicio = time.time()
svc.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

y_pred_svc = svc.predict(X_test_A_preprocessed)

tiempo_svc_a = fin - inicio

cm_svc = confusion_matrix(y_test_A, y_pred_svc)

### Support Vector Machines Optimizado

In [ ]:
svc_opt = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=svc_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_A_preprocessed, y_train_A)
fin = time.time()

# Extraer mejor modelo
best_model_svc = grid_search.best_estimator_

print(grid_search.best_params_)

# Predecir en el conjunto de prueba
y_pred_svc_opt = best_model_svc.predict(X_test_A_preprocessed)

tiempo_svc_opt_a = fin - inicio

cm_svc_opt = confusion_matrix(y_test_A, y_pred_svc_opt)

### MLP

In [ ]:
mlp = MLPClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
mlp.fit(X_train_A_preprocessed, y_train_A)

# Predecir en el conjunto de prueba
y_pred = mlp.predict(X_test_A_preprocessed)

# Evaluar el rendimiento
print("Matriz de confusión:")
print(confusion_matrix(y_test_A, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test_A, y_pred, digits=4))

### MLP Optimizado

In [ ]:
import itertools
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier

hidden_layer_options = [
    (50,),
    (100,),
    (100, 50),
    (100, 50, 25),
    (150, 75),
    (200, 100, 50)
]

activation_options = ['relu', 'tanh']
solver_options = ['adam']
learning_rates = [0.001, 0.0005, 0.0001]

# -------------------------------------------------------------------
# BÚSQUEDA DE TODAS LAS COMBINACIONES
# -------------------------------------------------------------------
best_f1 = 0
best_params = None

total_tests = len(hidden_layer_options) * len(activation_options) * len(solver_options) * len(learning_rates)
print(f"\nProbando {total_tests} combinaciones...\n")

for hls, act, solv, lr in itertools.product(hidden_layer_options, activation_options, solver_options, learning_rates):

    print(f"Probando: hidden={hls}, activation={act}, solver={solv}, lr={lr}")

    mlp = MLPClassifier(
        hidden_layer_sizes=hls,
        activation=act,
        solver=solv,
        learning_rate_init=lr,
        max_iter=500,
        random_state=42
    )

    mlp.fit(X_train_A_preprocessed, y_train_A)
    y_pred = mlp.predict(X_test_A_preprocessed)
    f1 = f1_score(y_test_A, y_pred, pos_label=1)  # F1 para la clase 1

    print(f" → F1 (clase 1): {f1:.4f}\n")

    if f1 > best_f1:
        best_f1 = f1
        best_params = (hls, act, solv, lr)

# -------------------------------------------------------------------
# RESULTADOS FINALES
# -------------------------------------------------------------------
print("========================================")
print("MEJOR COMBINACIÓN ENCONTRADA:")
print("Hidden layers:", best_params[0])
print("Activation:", best_params[1])
print("Solver:", best_params[2])
print("Learning rate:", best_params[3])
print(f"F1 (clase 1): {best_f1:.4f}")
print("========================================")


In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(150, 75),
    activation='tanh',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_A_preprocessed, y_train_A)

y_pred_mlp_opt = mlp.predict(X_test_A_preprocessed)

print("=== Classification Report ===")
print(classification_report(y_test_A, y_pred))

# Matriz de confusión
cm_mlp_opt = confusion_matrix(y_test_A, y_pred)

sns.heatmap(cm_mlp_opt, annot=True, cmap="Blues", fmt="d")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión MLP")
plt.show()

## 🟥 Escenario B: `Riesgo_Sobregiro`

### Regresión logística

In [ ]:
inicio = time.time()
rl.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

# Realiza predicciones
y_pred_lr_b = rl.predict(X_test_B_preprocessed)

tiempo_rl_b = fin - inicio

# Calcula la matriz de confusión
cm_lr_b = confusion_matrix(y_test_B, y_pred_lr_b)

### Árbol de decisión optimizado

In [ ]:
# Definir el diccionario de parámetros a probar
param_grid = {
    'max_depth': [None, 3, 5, 7, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=tree_optimizado,
    param_grid=param_grid,
    cv=5,  # validación cruzada con 5 particiones
    scoring='recall',
    n_jobs=-1  # usa todos los núcleos disponibles para acelerar
)

# Entrenar el GridSearchCV
inicio = time.time()
grid_search.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

print(grid_search.best_params_)

# Obtener el mejor modelo
best_model_dt = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_dt_opt_b = best_model_dt.predict(X_test_B_preprocessed)

tiempo_dt_b = fin - inicio

cm_dt_opt_b = confusion_matrix(y_test_B, y_pred_dt_opt_b)

### Random Forest

In [ ]:
inicio = time.time()
random_tree.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

y_pred_rf_b = random_tree.predict(X_test_B_preprocessed)

tiempo_rt_b = fin - inicio

cm_dt_b = confusion_matrix(y_test_B, y_pred_rf_b)

### Random Forest Optimizado

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=random_tree_opt,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_rf = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_rf_opt_b = best_model_rf.predict(X_test_B_preprocessed)

tiempo_rf_opt_b = fin - inicio

cm_rf_opt_b = confusion_matrix(y_test_B, y_pred_rf_opt_b)

### KNN

In [ ]:
knn = KNeighborsClassifier()

inicio = time.time()
knn.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

y_pred_knn_b = knn.predict(X_test_B_preprocessed)

tiempo_knn_b = fin - inicio

cm_knn_b = confusion_matrix(y_test_B, y_pred_knn_b)

### KNN Optimizado

In [ ]:
param_grid = {
    'n_neighbors': [1, 5, 10, 15, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=knn_opt,
    param_grid=param_grid,
    cv=5,
    scoring='recall',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_knn = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_knn_opt_b = best_model_knn.predict(X_test_B_preprocessed)

tiempo_knn_opt_b = fin - inicio

cm_knn_opt = confusion_matrix(y_test_B, y_pred_knn_opt_b)

### Naive Bayes

In [ ]:
inicio = time.time()
nb.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

y_pred_nb_b = nb.predict(X_test_B_preprocessed)

tiempo_nb_b = fin - inicio

cm_nb_b = confusion_matrix(y_test_B, y_pred_nb_b)

### Naive Bayes Optimizado

In [ ]:
nb_opt = GaussianNB()

param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=nb_opt,
    param_grid=param_grid,
    cv=5,
    scoring='recall',
    n_jobs=-1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

print(grid_search.best_params_)

# Extraer mejor modelo
best_model_nb = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred_nb_opt_b = best_model_nb.predict(X_test_B_preprocessed)

tiempo_nb_opt_b = fin - inicio

cm_nb_opt_b = confusion_matrix(y_test_B, y_pred_nb_opt_b)

### Support Vector Machines

In [ ]:
inicio = time.time()
svc.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

y_pred_svc_b = svc.predict(X_test_B_preprocessed)

tiempo_svc_b = fin - inicio

cm_svc = confusion_matrix(y_test_B, y_pred_svc_b)

### Support Vector Machines Optimizado

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Crear Grid Search
grid_search = GridSearchCV(
    estimator=svc_opt,
    param_grid=param_grid,
    cv=5,
    scoring='recall',
    n_jobs=1,
    verbose=2
)

# Ajustar modelo
inicio = time.time()
grid_search.fit(X_train_B_preprocessed, y_train_B)
fin = time.time()

# Extraer mejor modelo
best_model_svc = grid_search.best_estimator_

print(grid_search.best_params_)

# Predecir en el conjunto de prueba
y_pred_svc_opt_b = best_model_svc.predict(X_test_B_preprocessed)

tiempo_svc_opt_b = fin - inicio

cm_svc_opt_b = confusion_matrix(y_test_B, y_pred_svc_opt_b)

### MLP

In [ ]:
mlp = MLPClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
mlp.fit(X_train_A_preprocessed, y_train_A)

# Predecir en el conjunto de prueba
y_pred = mlp.predict(X_test_A_preprocessed)

# Evaluar el rendimiento
print("Matriz de confusión:")
print(confusion_matrix(y_test_A, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test_A, y_pred, digits=4))

### MLP Optimizado

In [ ]:
import itertools
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier

hidden_layer_options = [
    (50,),
    (100,),
    (100, 50),
    (100, 50, 25),
    (150, 75),
    (200, 100, 50)
]

activation_options = ['relu', 'tanh']
solver_options = ['adam']
learning_rates = [0.001, 0.0005, 0.0001]

# -------------------------------------------------------------------
# BÚSQUEDA DE TODAS LAS COMBINACIONES
# -------------------------------------------------------------------
best_f1 = 0
best_params = None

total_tests = len(hidden_layer_options) * len(activation_options) * len(solver_options) * len(learning_rates)
print(f"\nProbando {total_tests} combinaciones...\n")

for hls, act, solv, lr in itertools.product(hidden_layer_options, activation_options, solver_options, learning_rates):

    print(f"Probando: hidden={hls}, activation={act}, solver={solv}, lr={lr}")

    mlp = MLPClassifier(
        hidden_layer_sizes=hls,
        activation=act,
        solver=solv,
        learning_rate_init=lr,
        max_iter=500,
        random_state=42
    )

    mlp.fit(X_train_A_preprocessed, y_train_A)
    y_pred = mlp.predict(X_test_A_preprocessed)
    f1 = f1_score(y_test_A, y_pred, pos_label=1)  # F1 para la clase 1

    print(f" → F1 (clase 1): {f1:.4f}\n")

    if f1 > best_f1:
        best_f1 = f1
        best_params = (hls, act, solv, lr)

# -------------------------------------------------------------------
# RESULTADOS FINALES
# -------------------------------------------------------------------
print("========================================")
print("MEJOR COMBINACIÓN ENCONTRADA:")
print("Hidden layers:", best_params[0])
print("Activation:", best_params[1])
print("Solver:", best_params[2])
print("Learning rate:", best_params[3])
print(f"F1 (clase 1): {best_f1:.4f}")
print("========================================")

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(150, 75),
    activation='tanh',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_A_preprocessed, y_train_A)

y_pred_mlp_opt = mlp.predict(X_test_A_preprocessed)

print("=== Classification Report ===")
print(classification_report(y_test_A, y_pred))

# Matriz de confusión
cm_mlp_opt = confusion_matrix(y_test_A, y_pred)

sns.heatmap(cm_mlp_opt, annot=True, cmap="Blues", fmt="d")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión MLP")
plt.show()

---

## 🔟 **Evaluación de modelos**
Se analizan:
- Matriz de confusión  
- Classification report
- Tiempo de predicción

El objetivo es seleccionar el modelo más robusto para la segmentación de clientes

## 1️⃣1️⃣ **Variables más importantes del mejor modelo**

## Regresión lineal

In [ ]:
df_encoded2 = df.copy()

df_encoded2 = df_encoded2.drop(columns=['pdays', 'contact', 'day', 'duration', 'campaign']) #, 'Patrimonio_Alto', 'Riesgo_Sobregiro'

df_encoded2 = pd.get_dummies(df_encoded2, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'month', 'poutcome', 'subscribed_term_deposit'], drop_first=True)

In [ ]:
X = df_encoded2.drop(columns=['balance']) #'Patrimonio_Alto', 'Riesgo_Sobregiro', 
y = df_encoded2['balance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelos lineales

## Regresión lineal simple

In [ ]:
# Entrenar modelo de regresión lineal
lr_model = LinearRegression()

inicio = time.time()
lr_model.fit(X_train, y_train)
fin = time.time()

lr_time = fin - inicio

print("Tiempo LinearRegression:")
print(f"{lr_time:.2f}\n")

# Predicciones
y_pred = lr_model.predict(X_test)

# Evaluar modelo para train y para test
print("Evaluación del modelo de Regresión Lineal:")
lr_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
lr_r2 = r2_score(y_test, y_pred)
print(f"RMSE: {lr_rmse:.2f}")
print(f"R²: {lr_r2:.4f}\n")

## Regularización Lasso

El modelo Lasso aplica una penalización L1 sobre los coeficientes, lo que no solo ayuda a controlar la complejidad del modelo, sino que también realiza selección automática de variables, reduciendo a cero aquellas que aportan poca información.

De este modo, permite simplificar el modelo sin perder interpretabilidad.

In [ ]:
# Crear y ajustar GridSearchCV para Lasso
lasso = Lasso(max_iter=50000)

alphas = np.logspace(-3, 1, 10)
param_grid = {'alpha': alphas}

grid_lasso = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')

inicio= time.time()
grid_lasso.fit(X_train, y_train)
fin = time.time()

lasso_time = fin - inicio

print("Tiempo Lasso:")
print(f"{lasso_time:.2f}")

# Mejor modelo Lasso
best_lasso = grid_lasso.best_estimator_
print(f"Mejor valor de alpha para Lasso: {grid_lasso.best_params_['alpha']:.6f}")

# Predicciones con el mejor modelo Lasso
lasso_test_pred = best_lasso.predict(X_test)

# Evaluar modelo
print("\nEvaluación del modelo Lasso:")
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_test_pred))
lasso_r2 = r2_score(y_test, lasso_test_pred)

print(f"RMSE: {lasso_rmse:.2f}")
print(f"R²: {lasso_r2:.4f}\n")

## Regularización Ridge

Implementaremos Ridge con búsqueda de hiperparámetros usando GridSearchCV para encontrar el mejor valor de alpha.

In [ ]:
# Definir parámetros para búsqueda
alphas = np.logspace(-3, 1, 10)
param_grid = {'alpha': alphas}

# Crear y ajustar GridSearchCV
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')

inicio = time.time()
grid_ridge.fit(X_train, y_train)
fin = time.time()

ridge_time = fin - inicio

print("Tiempo Ridge:")
print(f"{ridge_time:.2f}")

# Mejor modelo Ridge
best_ridge = grid_ridge.best_estimator_
print(f"Mejor valor de alpha para Ridge: {grid_ridge.best_params_['alpha']:.6f}")

# Predicciones con el mejor modelo Ridge
ridge_test_pred = best_ridge.predict(X_test)

# Evaluar modelo
print("\nEvaluación del modelo Ridge:")
ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_test_pred))
ridge_r2 = r2_score(y_test, ridge_test_pred)

print(f"RMSE: {ridge_rmse:.2f}")
print(f"R²: {ridge_r2:.4f}\n")

## Análisis de coeficientes e interpretación

Analizaremos los coeficientes del modelo Lasso para entender qué variables son las más importantes para predecir el balance y cuáles tienen mayor o menor impacto o son eliminadas por el modelo.

In [ ]:
# Obtener coeficientes del modelo Lasso
feature_names = X_train.columns

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': best_lasso.coef_
})

# Ordenar coeficientes por valor absoluto
coef_df['Abs_Coefficient'] = abs(coef_df['Coefficient'])
coef_df_sorted = coef_df.sort_values('Abs_Coefficient', ascending=False)

# Mostrar los 10 coeficientes más importantes
print("Top 10 variables más importantes según Lasso:")
print(coef_df_sorted.head(10))

# Contar variables eliminadas (coeficientes = 0)
zero_coef = len(coef_df[coef_df['Coefficient'] == 0])
print(f"\nNúmero de variables eliminadas por Lasso (coeficiente = 0): {zero_coef}")

# Visualizar coeficientes más importantes
plt.figure(figsize=(12, 6))
sns.barplot(data=coef_df_sorted.head(10), x='Coefficient', y='Feature')
plt.title('Top 10 variables más importantes (Lasso)')
plt.xlabel('Coeficiente')
plt.tight_layout()
plt.show()

## Árboles y Bagging

En esta sección nos enfocaremos en los modelos no lineales basados en árboles y su primer ensemble.

### Análisis de Overfitting en Árboles de Regresión
Este bloque de código entrena múltiples modelos DecisionTreeRegressor variando la profundidad máxima del árbol (max_depth) para analizar cómo afecta la complejidad del modelo al error de entrenamiento (train) y de prueba (test).

Objetivo:
- Evaluar el sobreajuste (overfitting) y subajuste (underfitting) observando la evolución del RMSE en función de la profundidad del árbol.
- Determinar visualmente la profundidad óptima que equilibra buen rendimiento en test sin sobreajustar los datos de entrenamiento.

In [ ]:
# DecisionTreeRegressor
max_depths = range(1, 18)
train_rmses = []
test_rmses = []

for depth in max_depths:
    tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    # Error en train
    y_pred_train = tree.predict(X_train)
    train_rmses.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    
    # Error en test
    y_pred_test = tree.predict(X_test)
    test_rmses.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))

# Graficamos los resultados
plt.figure(figsize=(12, 7))
plt.plot(max_depths, train_rmses, 'o-', label='Error de Entrenamiento (Train RMSE)')
plt.plot(max_depths, test_rmses, 'o-', label='Error de Prueba (Test RMSE)')
plt.xlabel('Profundidad del Árbol (max_depth)')
plt.ylabel('RMSE')
plt.title('Error del Árbol vs. Profundidad', fontsize=16)
plt.legend()
plt.grid(True)
plt.show()

## Optimización de Decision Tree con GridSearchCV

Este código realiza una búsqueda de hiperparámetros para un DecisionTreeRegressor usando GridSearchCV. Se prueban distintos valores de max_depth, min_samples_split y min_samples_leaf para encontrar la combinación que minimice el RMSE en validación cruzada. Finalmente, se evalúa el modelo óptimo en el conjunto de prueba.

In [ ]:
tree = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [3, 5, 6, 8],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)

# Entrenar la búsqueda
inicio = time.time()
grid_search.fit(X_train, y_train)
fin = time.time()

tree_time = fin - inicio
print("Tiempo Decision Tree (GridSearchCV):")
print(f"{tree_time:.2f}")

# Calcular el RMSE
mejor_modelo = grid_search.best_estimator_

print("Mejores parámetros del Decision Tree:")
print(grid_search.best_params_)

y_pred = mejor_modelo.predict(X_test)

# Evaluar modelo
tree_r2_final = r2_score(y_test, y_pred)
tree_rmse_final = np.sqrt(mean_squared_error(y_test, y_pred))

print("\nEvaluación del modelo Decision Tree (GridSearchCV):")
print(f"RMSE: {tree_rmse_final:.2f}")
print(f"R²: {tree_r2_final:.4f}\n")

## Optimización de Random Forest con GridSearchCV

Este código entrena un RandomForestRegressor buscando automáticamente la mejor combinación de hiperparámetros (n_estimators y max_depth) usando GridSearchCV. Se calcula el tiempo de entrenamiento y se evalúa el modelo final en el conjunto de prueba mediante RMSE y R².

In [ ]:
forest = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 8]
}

grid_search = GridSearchCV(
    estimator=forest,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,  # usa todos los núcleos disponibles
    verbose=0
)

inicio = time.time()
grid_search.fit(X_train, y_train)
fin = time.time()
forest_time = fin - inicio

# Calcular el RMSE
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

forest_r2_final = r2_score(y_test, y_pred)
forest_rmse_final = np.sqrt(mean_squared_error(y_test, y_pred))

resultados_rf = pd.DataFrame({
    "Modelo": ["Random Forest (GridSearchCV)"],
    "Tiempo Entrenamiento (s)": [forest_time],
    "Mejores Parámetros": [grid_search.best_params_],
    "RMSE Test": [forest_rmse_final],
    "R2 Test": [forest_r2_final]
})

print(resultados_rf)

## Optimización de Gradient Boosting con GridSearchCV

Este código entrena un Gradient Boosting buscando automáticamente la mejor combinación de hiperparámetros (n_estimators, learning_rate y max_depth) usando GridSearchCV. Se calcula el tiempo de entrenamiento y se evalúa el modelo final en el conjunto de prueba mediante RMSE y R².

In [ ]:
# --- 1. Modelo Gradient Boosting (con GridSearchCV) ---
print("Iniciando GridSearchCV para Gradient Boosting...")
print("Esto puede tardar varios minutos...")

# Definir el modelo
gb_model = GradientBoostingRegressor(random_state=42)

# Definir el grid de hiperparámetros
# (Es un grid pequeño para que no tarde una eternidad)
param_grid_gb = {
    'n_estimators': [100, 150, 200],         # Número de árboles
    'learning_rate': [0.03, 0.05, 0.1],       # Tasa de aprendizaje
    'max_depth': [3, 6, 8]                 # Profundidad máxima
}

# Configurar GridSearchCV
# Usamos scoring='neg_mean_squared_error' para optimizar RMSE
# O 'r2' para ser consistentes con tu búsqueda anterior
grid_gb = GridSearchCV(estimator=gb_model,
                       param_grid=param_grid_gb,
                       cv=5,  # 3-fold CV para ir más rápido (puedes usar 5)
                       scoring='r2',
                       n_jobs=-1, # Usar todos los procesadores
                       verbose=1)

# Medir el tiempo de entrenamiento
start_time = time.time()
grid_gb.fit(X_train, y_train)
end_time = time.time()

gb_time = end_time - start_time

print(f"\nTiempo de entrenamiento Gradient Boosting (GridSearchCV): {gb_time:.2f} segundos")
print(f"Mejores parámetros encontrados: {grid_gb.best_params_}")

# Obtener el mejor modelo
best_gb = grid_gb.best_estimator_

# --- 2. Evaluación del modelo Gradient Boosting ---

print("\nEvaluación del modelo Gradient Boosting (con mejores hiperparámetros):")

# Predicciones
gb_train_pred = best_gb.predict(X_train)
gb_test_pred = best_gb.predict(X_test)

# Evaluar
gb_test_r2 = r2_score(y_test, gb_test_pred)
gb_test_rmse = np.sqrt(mean_squared_error(y_test, gb_test_pred))

print(f"RMSE Test: {gb_test_rmse:.2f}")
print(f"R² Test: {gb_test_r2:.4f}\n")

## Tabla Maestra de Resultados

In [ ]:
# --- Recolección de Métricas ---
# Asumimos que 'lr_model', 'best_lasso', y 'best_gb' SÍ existen en memoria de tu script anterior.

# 1. Crear la lista de datos manualmente
# Esta es la forma más robusta de hacerlo, usando las variables de tu script.
metric_data = [
    # (Nombre, R², RMSE)
    ("Regresión Lineal", lr_r2, lr_rmse, lr_time),
    ("Ridge", ridge_r2, ridge_rmse, ridge_time),
    ("Lasso", lasso_r2, lasso_rmse, lasso_time),
    ("Decission Tree Optimizado", tree_r2_final, tree_rmse_final, tree_time),
    ("Random Forest Optimizado", forest_r2_final, forest_rmse_final, forest_time),
    ("Gradient Boosting Optimizado", gb_test_r2, gb_test_rmse, gb_time)
]

# 2. Construir el DataFrame
resultados = []

for nombre, r2, rmse, time in metric_data:
    resultados.append({
        "Modelo": nombre,
        "R² Test": r2,
        "RMSE Test": rmse,
        "Tiempo": time
    })

# --- Crear la Tabla Maestra ---
if resultados:
    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.set_index('Modelo')
    df_resultados = df_resultados.sort_values(by='R² Test', ascending=False)
    
    # Mostrar la tabla
    print("\n--- Tabla Maestra de Resultados (TEST set) ---")
    display(df_resultados.style.format({
        'R² Test': '{:.4f}',
        'RMSE Test': '{:,.2f}'
    }))
else:
    print("La lista 'metric_data' está vacía. Verifica las variables.")

Conclusiones de la tabla maestra

## Comparar las Importancias de Variables

Esta celda genera los tres gráficos de importancia de variables uno al lado del otro.


---

## 1️⃣2️⃣ **Conclusiones**
- Principales hallazgos del EDA  
- Interpretación financiera de los resultados  
- Mejor modelo encontrado y por qué  
- Limitaciones del dataset  
- Posibles mejoras o líneas futuras  